In [1]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle

from multiprocessing import Process
from multiprocessing import Pool

In [1]:
#starting date = 29 tir

In [76]:
r = requests.get("https://jobvision.ir/jobs/489870/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%D8%AA%D8%A7%DB%8C%D9%BE%D8%B3%D8%AA-%D8%AD%D9%88%D8%B2%D9%87-%DB%8C-%DA%A9%D8%AA%D8%A7%D8%A8---%D8%AE%D8%A7%D9%86%D9%85?row=61&pageSize=30&searchId=382527027773765237&score=17.4&ReferrerJobPosition=8")
soup = BeautifulSoup(r.text,'lxml')

In [36]:
all_details = {}

In [37]:
import pickle

# Load the BeautifulSoup object
with open('contents_2', 'rb') as f:
    contents = pickle.load(f)

In [38]:
index = 0

for key,value in contents.items():
    
    details = {}
    soup = BeautifulSoup(value,'html.parser')
    try:
        # Title of the job ad
        title = soup.find("div",class_="text-black font-size-1 font-weight-bold py-2")
        name = title.contents[0]
        time = title.find("span").text
        details["نام آگهی"]  = name
        details["زمان"] = time
    except:
        details["نام آگهی"]  = None
        details["زمان"] = None


    # Location where job posted for
    try:
        location = soup.find("div",class_="row align-items-center col text-muted justify-content-end px-0").text
        details["مکان"] = location
    except:
        details["مکان"] = None
    # Salary information
    try:
        salary = soup.find("div",class_="col-12 col-lg d-flex justify-content-end align-items-end px-0 flex-column")
        salary = salary.contents[0].text.strip()
        details["حقوق"] = salary
    except:
        details["حقوق"] = None

    # Small information such as active hours , ...
    try:
        small_info = soup.find("div",class_="div col-12 row px-2 py-4")

        details["روز و ساعت کاری"] = small_info.contents[0].contents[1].text.strip()
        details["سفرهای کاری"] = small_info.contents[1].contents[1].text.strip()
        details["مزایا و تسهیلات"] = small_info.contents[2].contents[1].text.strip()
    except:
        details["روز و ساعت کاری"] = None
        details["سفرهای کاری"] = None
        details["مزایا و تسهیلات"] = None

    # Company Size , Company Type , ...
    try:
        
        divs = soup.find_all('div', {'class': 'col-4 mb-3 ng-tns-c184-6'})

        for div in divs:
            label = div.find('label', {'class': 'text-black font-weight-bold ng-tns-c184-6'})
            if label is not None:
                field = label.text.strip()
                value_div = div.find('div', {'class': 'text-muted ng-tns-c184-6'})
                if value_div is not None:
                    value = value_div.text.strip()
                    details[field] = value
    except:
        pass

    # Requirements
    try:
        requirements = soup.find("div",class_="col-12 row px-0 mb-4")
        requirements = requirements.find("div",class_="col-12 row px-0")

        requirements = requirements.findAll('div', {'class': 'col-12 row align-items-center justify-content-between px-0 mb-3 ng-star-inserted'})


        for requirement in requirements:
            # Extract the title and value divs
            title_div = requirement.find('div', {'class': 'requirement-title row align-items-center font-weight-bold py-2 px-2 bg-light'})
            value_div = requirement.find('div', {'class': 'requirement-value'})

            title = title_div.text.strip()

            # Check if the value div contains any tags
            tags = value_div.find_all('app-tag')
            if tags:
                # If it does, extract the tag values and store them in a list
                values = [tag.text.strip() for tag in tags]
            else:
                # If it doesn't, extract the text as usual
                values = value_div.text.strip()

            details[title] = values
    except:
        pass
    all_details[index] = details
    index += 1
    print(index,end="\r")

In [39]:
df = pd.DataFrame.from_dict(all_details).T

In [40]:
df

,نام آگهی,زمان,مکان,حقوق,روز و ساعت کاری,سفرهای کاری,مزایا و تسهیلات,اندازه سازمان,صنعت,سن,...,زبان,تحصیلات,Company Size,Industry,Gender,Education,Language,Software,Military service,Age
0,کارگر ساده - آقا,(64 روز پیش),رباط کریم,12 - 10 میلیون تومان,شنبه تا چهارشنبه 8 تا 17:33، پنجشنبه 8:30 تا ...,-,پاداش -بیمه درمان تکمیلی,51 تا 200 نفر,تولیدی / صنعتی,23 - 45 سال,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,کارشناس موشن گرافیست,(79 روز پیش),تهران، آرژانتین,,شنبه تا چهارشنبه 7 الی 16,-,وام -پاداش -بیمه درمان تکمیلی -تسهیلات تفری...,بیش از 5000 نفر,بانکداری,تا 35 سال,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,مدیر اجرایی,(78 روز پیش),تهران، جردن,20 - 15 میلیون تومان,10 تا 20,-,پورسانت,11 تا 50 نفر,سرمایه گذاری و مالی,25 - 43 سال,...,[انگلیسی| متوسط ۵۰٪],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,کارشناس مرکز تماس ( شیفت شب) - آقا,(22 روز پیش),کرج، حیدرآباد,,شیفت شب ( 3 روز در هفته ),-,بیمه درمان تکمیلی -اتاق بازی -ناهار -میان و...,501 تا 1000 نفر,سرمایه گذاری و مالی,20 - 30 سال,...,[انگلیسی| متوسط ۵۰٪],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,کارشناس تحلیل و تست نرم افزار,(51 روز پیش),قم، عمار یاسر,,7:30 تا 17 شنبه تا چهارشنبه و پنجشنبهها از ساع...,-,وام -پاداش -پورسانت -بیمه درمان تکمیلی -کم...,201 تا 500 نفر,تولیدی / صنعتی,25 - 35 سال,...,[انگلیسی| پایین تر از متوسط ۳۰٪],[کارشناسی| کامپیوتر / فناوری اطلاعات],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,کارگر ساده - آقا,(42 روز پیش),شیراز,,شنبه تا چهارشنبه 8 تا 16:30 - پنج شنبه ها 8 تا 12,-,-,11 تا 50 نفر,تولیدی / صنعتی,25 - 40 سال,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
596,سرپرست برنامه ریزی تامین,(74 روز پیش),تهران، شهران,,شنبه تا چهارشنبه: 8 الی 16:30، پنج شنبه ها یک ...,-,وام -پاداش -بیمه درمان تکمیلی -بسته ها و هد...,51 تا 200 نفر,تولیدی / صنعتی,NaN,...,[انگلیسی| بالاتر از متوسط ۷۰٪],[کارشناسی ارشد| مهندسی صنایع],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597,کارشناس توسعه کسب و کار,(29 روز پیش),تهران، فردوسی,,شنبه تا چهارشنبه 08:30 الی 17:30,بر حسب نیاز,بیمه درمان تکمیلی,1001 تا 5000 نفر,حمل و نقل / ترابری,25 - 35 سال,...,[انگلیسی| متوسط ۵۰٪],[کارشناسی| مهندسی صنایع],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
598,پشتیبان سخت افزار و نرم افزار,(40 روز پیش),کرج، هفت تیر,,شنبه تا چهارشنبه 9 الی 17/30 و پنج شنبه 9 الی 13,به فراخور کاری و صلاحدید مدیر مربوطه,وام -پاداش -پورسانت -بیمه درمان تکمیلی -بن...,51 تا 200 نفر,فناوری اطلاعات / نرم افزار و سخت افزار,25 - 35 سال,...,[انگلیسی| متوسط ۵۰٪],[کارشناسی| کامپیوتر / فناوری اطلاعات],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df.to_csv("test.csv",index=False)